This code is designed to function like the "analysis" notebook in the other code, but with most of the actual "functions" defined in external python files in an effort to make the actual analysis look cleaner.

In [2]:
#### Imports
import cv2, json
from matplotlib import pyplot as plt
%matplotlib qt  

import numpy as np
from lmfit import report_fit
import pandas as pd
import sys
from scipy.optimize import leastsq
from scipy.optimize import curve_fit

from traj import load, link, filter_trajlen, get_prop, sort_traj
from funcs import acorr, fourier, sinfit, expfit, phaseplot
from flash import load_intensity, filt_d1, test_filter, detect, intersect, INTERSECT, report_ivl


In [3]:
#### load intensities from json files into lists
p1 = load_intensity('3pt708_vpp7.json', path='march11_h2')
p2 = load_intensity('3pt708_vpp7_2.json', path='march11_h2')
p3 = load_intensity('3pt708_sweep.json', path='march11_h2')



#### Test the filter
# test_filter(p3, filt=filt_d1, thresh=0.7)

#### Detect the flashes
thresh=0.75
P_bool = [detect(filt_d1(p), thresh=thresh) for p in [p1, p2, p3]]   #### Detect flashes at given threshold     
FLASH = [np.arange(np.size(fl))[fl] for fl in P_bool]                #### Indices where flash detected
DARK = [np.arange(np.size(fl))[~fl] for fl in P_bool]                #### Indices where flash not detected

print(FLASH[0])

FLASH[2][-3] = FLASH[2][0]
# ivl1 = INTERSECT(FLASH[2], f=3.708)
# print(ivl1[1] - ivl1[0])
# report_ivl(ivl1, FL=FLASH[2])



[ 47  55 136 144 225 233 314 322 403 411 492 500 581 589 670 678 767 848
 856]


In [4]:
#### Load trajectories from json files into dataframes
def quickload(filename, scatter=False, path=''):
    dat = link( load(filename, path=path, refined=False) )
    filter_trajlen(dat)
    
#     print(dat[{'x_p', 'particle'}])
    sort_traj(dat)
#     print(dat[{'x_p', 'particle'}])
    if(scatter):
        X = get_prop(dat, 'x_p')
        Y = get_prop(dat, 'y_p')
        for i in range(len(X)):
            plt.scatter(X[i], Y[i], label='traj. {} ({} points)'.format(i, np.size(X[i])))
        plt.legend()
    return dat

d0 = quickload('nofield.json', scatter=True, path='march11_h2')
d1 = quickload('3pt708_vpp7.json', scatter=True, path='march11_h2')
d2 = quickload('3pt708_vpp7_2.json', scatter=True, path='march11_h2')
d3 = quickload('3pt708_sweep.json', scatter=True, path='march11_h2')


# d00 = quickload('nofield1.json', scatter=True, path='five')
# d01 = quickload('nofield2.json', scatter=True, path='five')
# d02 = quickload('nofield3.json', scatter=True, path='five')

# d10 = quickload('3pt078hz_vpp7_1.json', scatter=True, path='five')
# d11 = quickload('3pt078hz_vpp7_2.json', scatter=True, path='five')
# d12 = quickload('3pt078hz_vpp7_3.json', scatter=True, path='five')

# d20 = quickload('3pt078hz_vpp10_1.json', scatter=True, path='five')
# d21 = quickload('3pt078hz_vpp10_2.json', scatter=True, path='five')

Frame 1310: 5 trajectories present.


In [5]:
#### Specify which runs to analyze and how many particles are in each run; get trajectory data from dataframes
runs =[d0, d1, d2, d3]
dim = 5    ## Keep the first 'dim' trajectories

X, Y, R = [ [get_prop(runs[i], prop) for i in range(len(runs))] for prop in ['x_p', 'y_p', 'r']]
X0, Y0, R0 = [ [get_prop(runs[i], prop) for i in range(len(runs))] for prop in ['x_p0', 'y_p0', 'r0']]

# # X, Y, R = runs2props(runs, ['x_p', 'y_p', 'r'])
# # X0, Y0, R0 = runs2props(runs, ['x_p0', 'y_p0', 'r0'])

# for x in X:
#     plt.figure()
#     for xi in x:
#         plt.plot(xi)

A = [ [np.mean(a) for a in get_prop(runs[i], 'a_p')] for i in range(len(runs))] 
print(A)
print('')
L = [ [(np.mean(x[i+1]) - np.mean(x[i]))*0.048 for i in range(dim-1)] for x in X]
# print([l*0.048 for l in L])
print(L)
# for x in X:
#     L = [np.mean(x[i+1]) - np.mean(x[i]) for i in range(dim-1)]
#     print(L)
#     print(np.mean(L))
#     print(' ')
    
    

[[1.1321625799883153, 1.97903124157709, 1.828955548280198, 0.9093165611492768, 0.4740024128991749], [1.6278176248395766, 1.8237418330682291, 1.9076703844843683, 1.4844573875375695, 0.844462364509299], [1.5609569929648137, 2.036631678469515, 2.3274582055743767, 1.5226487877288302, 0.9558801277392153], [2.065206018865427, 1.2973215764119914, 1.4935409639920105, 1.826215208258727, 1.1346110405583676]]

[[11.887156519699515, 11.927622920145513, 11.981864777509884, 11.985090823507358], [11.852335613782728, 11.969583097550677, 12.002842442989865, 11.948105805532096], [12.20960943647744, 11.946225334720337, 11.938661858157834, 12.002784101845563], [11.93426456072565, 11.918398652030891, 11.960515388122143, 12.012321096878571]]


In [6]:
#### Compute normal modes

#### Return the matrix (gamma*H) = C/|i-j|; C=(3a/2L)
# def H0(dim, C=1.5*0.25):
C=0.25*np.mean(A)/(np.mean(L))
print('a/L ~ {}'.format(C))
def H0(dim, C=1.5*C):    ## 0.048 microns/pixel
    H = np.ones([dim, dim])
    for i in range(dim):
        for j in range(dim):
            if i is not j:
                H[i, j] = C/abs(i-j)
                H[j, i] = C/abs(i-j)
    return H
LBDA, XI = np.linalg.eig( H0(dim) )
for i in range(dim):
    print('{}: lambda = {:.2f} | xi = {}'.format(i,LBDA[i], XI[i]))
    
#### Compute normal modes using matrix multiplication
def getNormalModes(vals, XI):
    out = []
    count = 0
    for x in vals:
        count += 1
        out.append(0*x)
    for i in range(count):
        for j in range(count):
            out[i] += XI[i][j]*vals[j]
            
    return out

#### Compute Normal Modes
Xn = [getNormalModes(X[i], XI) for i in range(len(runs))]
Yn = [getNormalModes(X[i], XI) for i in range(len(runs))]

Rn, X0n, Y0n, R0n = [[ getNormalModes(PROP[i], XI) for i in range(len(runs))] for PROP in [R, X0, Y0, R0]]

a/L ~ 0.031577712370284484
0: lambda = 1.12 | xi = [-0.37678755 -0.54412511 -0.54468172 -0.45158373  0.24769532]
1: lambda = 1.01 | xi = [-0.47788678 -0.45158373  0.09778469  0.54412511 -0.51192029]
2: lambda = 0.97 | xi = [-5.09225627e-01  6.98367751e-16  6.22511013e-01  1.58554837e-14
  5.94280489e-01]
3: lambda = 0.95 | xi = [-0.47788678  0.45158373  0.09778469 -0.54412511 -0.51192029]
4: lambda = 0.94 | xi = [-0.37678755  0.54412511 -0.54468172  0.45158373  0.24769532]


In [15]:


XIm = np.linalg.inv(XI)
k = [1/1.745, 1/2.555, 1/4.453, 1/4.524, 1/8.380]
K = np.zeros([dim, dim])
for i in range(dim):
    K[i][i] = XI[i][i]
print(np.matmul(XI, K))
print(np.matmul(K, XI))
print('')
print(np.matmul(XI, K) - np.matmul(K, XI))
print('')
Hk = np.matmul(np.matmul(XI, K), XIm)
print(Hk)
LBDA2, XI2 = np.linalg.eig( Hk )
for i in range(dim):
    print('{}: lambda = {:.2f} | xi = {}'.format(i,LBDA2[i], XI2[i]))
    

[[ 1.41968856e-01  2.45718047e-01 -3.39070369e-01  2.45718047e-01
   6.13529696e-02]
 [ 1.80061788e-01  2.03927866e-01  6.08720435e-02 -2.96072134e-01
  -1.26800257e-01]
 [ 1.91869875e-01 -3.15371515e-16  3.87519962e-01 -8.62736680e-15
   1.47200493e-01]
 [ 1.80061788e-01 -2.03927866e-01  6.08720435e-02  2.96072134e-01
  -1.26800257e-01]
 [ 1.41968856e-01 -2.45718047e-01 -3.39070369e-01 -2.45718047e-01
   6.13529696e-02]]
[[ 1.41968856e-01  2.05019565e-01  2.05229289e-01  1.70151127e-01
  -9.33285107e-02]
 [ 2.15805895e-01  2.03927866e-01 -4.41579730e-02 -2.45718047e-01
   2.31174874e-01]
 [-3.16998561e-01  4.34741616e-16  3.87519962e-01  9.87021324e-15
   3.69946149e-01]
 [ 2.60030196e-01 -2.45718047e-01 -5.32071025e-02  2.96072134e-01
   2.78548682e-01]
 [-9.33285107e-02  1.34777241e-01 -1.34915110e-01  1.11855175e-01
   6.13529696e-02]]

[[ 0.00000000e+00  4.06984815e-02 -5.44299657e-01  7.55669201e-02
   1.54681480e-01]
 [-3.57441073e-02  0.00000000e+00  1.05030017e-01 -5.03540870e

In [29]:
print(1/np.matmul(XI, np.matmul(k, XIm)))
print(1/np.matmul(np.matmul(XI, k), XIm))


[  1.67721259   3.18243748   3.72558641   3.85062005 -26.65427458]
[  1.67721259   3.18243748   3.72558641   3.85062005 -26.65427458]


In [6]:
#### Compute autocorrelation functions 
cX = [ [acorr(a) for a in X[i]] for i in range(len(runs))] 
cY, cR, cX0, cY0, cR0 = [[ [acorr(a) for a in PROP[i]] for i in range(len(runs))] for PROP in [Y, R, X0, Y0, R0]]

cXn, cYn, cRn, cX0n, cY0n, cR0n = [[ [acorr(a) for a in PROP[i]] for i in range(len(runs))] for PROP in [Xn, Yn, Rn, X0n, Y0n, R0n]]

In [187]:
######## Plots a (rows)x(cols) dimensional grid of subfigures  
#### INPUT:   VALS: 2d list of dimension (cols)x(rows) of data   
####          FUNCS: 1d list of size (cols) of functions (see below)
####          PARAMS: (optional) list of (cols) lists of parameters to pass to functions
#### 
#### FUNCTIONS are of the form "func(x, params=None)" that 1) plot something and 2) return something of same shape as input x
#### OUTPUT:   a 2d list of dimension (cols)x(rows) where OUT[i][j] = FUNCS[i](VALS[i][j], PARAMS[i]);
####      AND, plots a (cols)x(rows) dim. grid of plots where subfigure where column i, row j is a plot of OUT[i][j]
####
#### optional: val_labels -> list of (rows) labels for each row; func_labels -> list of (cols) labels for each column

# def plotArray(VALS, FUNCS, PARAMS=None, val_labels=None, func_labels=None):
#     cols = len(FUNCS)
#     func_labels = ['func_{}'.format(i) for i in np.arange(cols)+1] if func_labels is None else func_labels
#     val_labels = ['func_{}'.format(i) for i in np.arange( len(VALS[0])) + 1] if val_labels is None else val_labels
    
#     OUT = []
#     for i in range(cols):
#         func = FUNCS[i]
#         vals = VALS[i]
#         rows = len(vals)
        
#         out = []
#         for j in range(rows):
#             plt.sublot(rows, cols, (j*cols) + i+1)
#             out.append(func(vals[j]))
#             if i is 0:
#                 plt.ylabel(val_labels[j])
#             if j is 0:
#                 plt.title(func_labels[i])
                
# def raw(x, params=None):
#     plt.plot(x)



def title_array(titles, rows):
    for j in range(len(titles)):
        plt.subplot(rows, len(titles), j+1)
        plt.title(titles[j])

def xlabel_array(xlabels, rows):
    for j in range(len(xlabels)):
        plt.subplot(rows, len(xlabels), (rows-1)*len(xlabels) + j+1)
        plt.xlabel(xlabels[j])

def ylabel_array(ylabels, cols):
    for j in range(len(ylabels)):
        plt.subplot(len(ylabels), cols, j*cols+1)
        plt.ylabel(ylabels[j])    
        
        
        
def plotCol(vals, cols, index, FPS=30):
    rows = len(vals)
    t = np.arange(len(vals[0]))/FPS
    for i in range(rows):
        plt.subplot(rows, cols, (i*cols) + index)
        plt.plot(t, vals[i])

def plotCol_fourier(vals, cols, index, FPS=30):
    rows = len(vals)
    out = []
    for i in range(rows):
        plt.subplot(rows, cols, (i*cols) + index)
#         plt.ylabel('Particle {}'.format(i+1))
        hz, fx = fourier(vals[i], FPS=FPS)
        plt.plot(hz, fx)
#         print('{}: Peak at {} hz'.format(i, hz[np.where(fx==max(fx))[-1]]))
#         print('')
        out.append(fx)
    return out

def plotCol_phase(vals, cols, index, FL=None, f=6.14, dt=20/1000, FPS=30):
    rows = len(vals)
    t = np.arange(len(vals[0]))/FPS
    Amp = []
    Freq = []
    Phase = []
    phi0 = np.array(FL[0])/(2*np.pi*f) if FL is not None else [] ## interval of first LED flash, if included
    
    for i in range(rows):
        plt.subplot(rows, cols, (i*cols) + index)
        offset = np.mean(vals[i])
        params, X = sinfit(vals[i] - offset, k0=f)                        ## Fit to a sine wave
        X += offset
        Amp.append(params[0])
        Freq.append(params[1])
        Phase.append(params[2])
        str1 = '{}: FIT: A={:.3f}, f={:.3f}, phi={:.3f}'.format(i, params[0], params[1], params[2])
        
        color=next(plt.gca()._get_lines.prop_cycler)['color']    ## Plot raw data and fit
        plt.plot(t, vals[i], color=color)
        plt.plot(t, X, linestyle=':', color=color)
        x0 = params[3]/(2*np.pi*f)                               ## Black lines at maxima
        while x0 < t[-1]:
            x0 = x0
#             plt.axvline(x=x0, color='black')
            x0 += 1/f      
        
        if FL is not None:        ## If flashes included, print green lines at interval and print full phase
            str2 = ' -> dphi ~ [{}, {}]'.format(params[2] - phi0[0], params[2] - phi0[1])
            print(str1 + str2)
            for ivl in FL:
                plt.axvline(x=ivl[0], color='green')
                plt.axvline(x=ivl[1], color='green')
        else:                     ## Otherwise, just print fit output
            print(str1)
    print('')
    return Amp, Freq, Phase, phi0

def plotCol_cos(vals, cols, index, f=6.14, A=1.0, FPS=30):
    rows = len(vals)
    t = np.arange(len(vals[0]))/FPS
    Amp = []
    Freq = []
    Phase = []
    for i in range(rows):
        plt.subplot(rows, cols, (i*cols) + index)
        params, X = sinfit(vals[i], k0=f, guess=[A, f, np.pi/2])                        ## Fit to a sine wave
        params[2] = params[2] + np.pi/2
        Amp.append(params[0])
        Freq.append(params[1])
        Phase.append(params[2])
        print('{}: FIT: A={:.3f}, f={:.3f}, phi={:.3f}'.format(i, params[0], params[1], params[2]))
        
        color=next(plt.gca()._get_lines.prop_cycler)['color']    ## Plot raw data and fit
        plt.plot(t, vals[i], color=color)
        plt.plot(t, X, linestyle=':', color=color)
    print('')
    return Amp, Freq, Phase



def plotCol_exp(vals, cols, index, n=25, FPS=30):
    rows = len(vals)
    t = np.arange(len(vals[0]))/FPS
    Amp = []
    Tau = []
    for i in range(rows):
        plt.subplot(rows, cols, (i*cols) + index)
        params, x = expfit(vals[i])
        Amp.append(params[0])
        Tau.append(params[1])
        str1 = 'C[0]={:.3f} | FIT: C[0]={:.3f}, tau={:.3f}'.format(vals[i][0], params[0], params[1])
        
        color=next(plt.gca()._get_lines.prop_cycler)['color']
        plt.plot(t[:n], vals[i][:n], color=color)
        plt.plot(t[:n], x[:n], linestyle=':', color=color)
        print(str1)
    print('')
    return Amp, Tau

def flash2ivls(FLASH, f=3.708, dt=20/1000, FPS=30):
    red_ivl = INTERSECT(FLASH, f=f, dt=dt, FPS=FPS)
    return [fl/FPS + red_ivl for fl in FLASH]
            

In [222]:
# i = 1
# cols=6
# plotCol(X0[i], cols, 1)
# plotCol_fourier(X0[i], cols, 2)
# plotCol_phase(X0[i], cols, 3, f=3.708)
# plotCol_phase(X0[i], cols, 4, f=3.708, FL=flash2ivls( FLASH[i-1], f=3.708 ))
# plotCol_exp(cX0[i], cols, 5)
# plotCol_exp(cX0[0], cols, 6)

# print('')
# print([np.mean(x) for x in X[i]])

i = 1
cols=5
plt.figure()

print('Trajectory')
Ad, fd, phid, phi0 = plotCol_phase(X[i], cols, 1, f=3.708, FL=flash2ivls( FLASH[i-1], f=3.708 ))

plotCol_fourier(X0[i], cols, 2)
C0, tau0 = plotCol_exp(cX0[0], cols, 3)
plotCol_exp(cX0[i], cols, 4)
Acorr, fcorr, phicorr = plotCol_cos([cX0[i][j][:25] - cX0[0][j][:25] for j in range(len(cX0)+1)], cols, 5, f=3.708)

title_array(['Trajectory', 'Fourier Transform', 'Autocorrelation', 'Acorr w/o field', 'Acorr - Accor w/o field'], 5)
plt.suptitle('Trajectory Analysis')

print('')
print('Normal Modes')

plt.figure()
An, fn, phin, trash = plotCol_phase(Xn[i], 1, 1, f=3.708)

plt.figure()
cols = 3
Cn, taun = plotCol_exp(cX0n[0], cols, 1)
plotCol_exp(cX0n[i], cols, 2)
Acos, fcos, phicos = plotCol_cos([cX0n[i][j][:25] - cX0n[0][j][:25] for j in range(len(cX0n)+1)], cols, 3, f=3.708)

title_array( ['Undriven', 'Driven', '(Driven) - (Undriven)'], 5)

plt.suptitle('Relaxation of Normal Modes')



# exparamsn = plotCol_exp(cX0n[0], cols, 4)
# plotCol_exp(cX0n[i], cols, 4)
# plotCol_fourier(cX0[i], cols, 4)


# cosparams = plotCol_cos([cX0n[i][j][:825] - cX0n[0][j][:825] for j in range(len(cX0n)+1)], cols, 5, f=3.708)

# titles = ['Trajectory', 'Fourier Transform', 'Autocorrelation', 'Acorr of Normal Modes', 'Acorr Norm Modes - Accor w/o field']
# for i in range(cols):
#     plt.subplot(len(X0[i]), cols, i+1)
#     plt.title(titles[i])
    
print('')
print(['{:.2f}'.format(np.mean(x)) for x in X[i]])
print('')

for j in range(len(X0[i])):
    print('{} | LBDA={:.3f} | nofield: C[0]={:.3f}, tau={:.3f} | sin: A={:.3f}, f={:.3f}, phi={:.3f}, tan(dphi)={:.3f} | cos: A={:.3f}, f={:.3f}, phi={:.3f}'.format(j, LBDA[j], exparams[j][0], exparams[j][1], sinparams[j][0], sinparams[j][1], sinparams[j][2], np.tan(sinparams[j][2] - np.mean(phi0)), cosparams[j][0], cosparams[j][1], cosparams[j][2] ))
print('')
## tau_j = gamma/(k*lambda_j) --> gamma = tau_j*k*lambda_j
## gamma = (k - m omega^2)tan(phi)/omega

for j in range(len(X0[i])):
    print('{}: gamma/k = tan(phi)/omega = {:.3f}'.format(j, np.tan(phid[j] + np.mean(phi0))/(2*np.pi*3.708)))
    print('{}: gamma/k = tau*lambda = {:.3f}'.format(j, LBDA[j]*tau0[j]))

print('')
## tau_j = gamma/(k*lambda_j) --> gamma = tau_j*k*lambda_j
## C[0] = kBT/k
for j in range(len(X0[i])):
    print('{}: kBT/k = C[0] = {:.3f}'.format(j, C0[j]))
    print(' : gamma/k = lambda*tau = {:.3f}'.format(LBDA[j]*tau0[j]))
    print(' : gamma/kBT = lambda*tau/C[0] = {:.3f}'.format(tau0[j]/C0[j]))
print('')


for j in range(len(X0[i])):
    print('{}: tau = gamma/(lambda*k) = (k*tanphi/omega)*(1/(lambda*k)) = tanphi/(lambda*2 pi f) = {:.3f}'.format(j, (1/(LBDA[j]))*np.tan(phid[j] + np.mean(phi0))/(2*np.pi*3.708)))
    print(' : gamma/k = lambda*tau = {:.3f}'.format(LBDA[j]*tau0[j]))
    print(' : gamma/kBT = lambda*tau/C[0] = {:.3f}'.format(tau0[j]/C0[j]))
print('')
k0 = [1/C0[j] for j in range(5)]
kn = [sum( [XI[i][j]*k0[j] for j in range(5)] ) for i in range(5)]

print(k0)
print(kn)

print([Acorr[j]/Ad[j]*k0[j] for j in range(5)])

print([Acorr[j]/(Ad[j]*k0[j]) for j in range(5)])

print([sum([])])
# for i in range(cols):
#     for j in range(dim):
#         plt.subplot(dim, cols, i*dim + 1)
#         plt.ylabel(XI[i])

print(phi0)
print(phid)
print(FLASH)


Trajectory
0: FIT: A=1.015, f=3.707, phi=1.332 -> dphi ~ [1.2651067453320846, 1.264549268069354]
1: FIT: A=1.356, f=3.708, phi=1.050 -> dphi ~ [0.9830632900077125, 0.9825058127449819]
2: FIT: A=1.897, f=3.707, phi=0.938 -> dphi ~ [0.8717045244431342, 0.8711470471804036]
3: FIT: A=2.101, f=3.708, phi=0.774 -> dphi ~ [0.7076550147035294, 0.7070975374407988]
4: FIT: A=2.957, f=3.708, phi=0.657 -> dphi ~ [0.5902981595614845, 0.5897406822987539]

C[0]=1.124 | FIT: C[0]=1.123, tau=0.021
C[0]=1.515 | FIT: C[0]=1.514, tau=0.015
C[0]=1.915 | FIT: C[0]=1.910, tau=0.021
C[0]=1.985 | FIT: C[0]=1.950, tau=0.035
C[0]=3.482 | FIT: C[0]=3.493, tau=0.038

C[0]=1.724 | FIT: C[0]=1.745, tau=0.029
C[0]=2.512 | FIT: C[0]=2.555, tau=0.027
C[0]=4.331 | FIT: C[0]=4.453, tau=0.027
C[0]=4.341 | FIT: C[0]=4.524, tau=0.029
C[0]=7.935 | FIT: C[0]=8.380, tau=0.030

0: FIT: A=0.510, f=3.776, phi=2.981
1: FIT: A=0.926, f=3.774, phi=2.980
2: FIT: A=1.884, f=3.700, phi=3.132
3: FIT: A=2.218, f=3.712, phi=3.115
4: FIT: 

In [202]:
plt.plot([abs(a) for a in Ad])
plt.plot([abs(a) for a in Acorr])

In [80]:
plt.scatter(range(len(X0)), [np.tan(sinparams[j][2] - np.mean(phi0))/(2*np.pi*3.708) for j in range(len(X0))])
plt.scatter(range(len(X0)), [LBDA[j]*exparams[j][1] for j in range(len(X0))])


In [153]:
i = 1
cols = 2
plotCol_phase(X[i], 2, 1, FL = flash2ivls( FLASH[i-1], f=3.708 ), f=3.708)
plotCol_fourier(X[i], 2, 2)

    
    
title_array(['x(t)', 'Fourier Transform'], 5)
    
xlabel_array(['sec', 'Hz'], 5)
    
ylabel_array([ 'particle {}'.format(j+1) for j in range(len(X0[i])) ], 2)

# plt.suptitle('Relaxation of Normal Modes')


[array([1.55065804, 1.56364617]), array([1.8173247 , 1.83031284]), array([4.5173247 , 4.53031284]), array([4.78399137, 4.7969795 ]), array([7.48399137, 7.4969795 ]), array([7.75065804, 7.76364617]), array([10.45065804, 10.46364617]), array([10.7173247 , 10.73031284]), array([13.4173247 , 13.43031284]), array([13.68399137, 13.6969795 ]), array([16.38399137, 16.3969795 ]), array([16.65065804, 16.66364617]), array([19.35065804, 19.36364617]), array([19.6173247 , 19.63031284]), array([22.3173247 , 22.33031284]), array([22.58399137, 22.5969795 ]), array([25.55065804, 25.56364617]), array([28.25065804, 28.26364617]), array([28.5173247 , 28.53031284])]
0: FIT: A=1.015, f=3.707, phi=1.332 -> dphi ~ [1.2651067453320846, 1.264549268069354]
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988133764789467
0.012988

In [171]:
i = 0
cols = 2
plotCol_exp(cX0[i], 2, 1)
plotCol_exp(cX0n[i] 2, 2)

titles = ['<x(0)x(t)>', '<xi(0)xi(t)>']
for j in range(cols):
    plt.subplot(len(X0[i]), cols, j+1)
    plt.title(titles[j])
    
xlabels = ['sec', 'sec']
for j in range(cols):
    plt.subplot(len(X0[i]), cols, (len(X0[i])-1)*cols + j+1)
    plt.xlabel(xlabels[j])
    
ylabels = ['particle {}'.format(j+1) for j in range(len(X0[i]))]
for j in range(len(X0[i])):
    plt.subplot(len(X0[i]), cols, j*cols+1)
    plt.ylabel(ylabels[j])

    
ylabels = ['j = {}'.format(j+1) for j in range(len(X0[i]))]
for j in range(len(X0[i])):
    plt.subplot(len(X0[i]), cols, j*cols+1)
    plt.ylabel(ylabels[j])
    
plt.suptitle('Autocorrelation')

print(LBDA)
for i in range(5):
    print('lbda={:.3f}'.format(LBDA[i]))


C[0]=1.868 | FIT: C[0]=1.484, tau=4956095606.648
C[0]=2.488 | FIT: C[0]=1.968, tau=1537074155.836
C[0]=3.723 | FIT: C[0]=2.916, tau=774.644
C[0]=5.084 | FIT: C[0]=4.135, tau=6263699063.882
C[0]=6.287 | FIT: C[0]=5.206, tau=187.264

C[0]=1.868 | FIT: C[0]=1.508, tau=16.164
C[0]=2.488 | FIT: C[0]=2.007, tau=9.944
C[0]=3.723 | FIT: C[0]=3.044, tau=10.530
C[0]=5.084 | FIT: C[0]=4.195, tau=14.175
C[0]=6.287 | FIT: C[0]=5.190, tau=14.659

[1.12395065 1.01436553 0.97343621 0.95010954 0.93813807]
lbda=1.124
lbda=1.014
lbda=0.973
lbda=0.950
lbda=0.938


In [199]:
plt.plot(np.arange(np.size(p2))/30, p2)
plt.title('Intensity')
plt.xlabel('sec')

Text(0.5, 0, 'sec')

In [208]:
for i in range(5):
    print('{:.3f}'.format(LBDA[i]))

1.124
1.014
0.973
0.950
0.938


In [18]:
class Particle(object):
    
    def __init__(self, name):
        
        

class System(object):
    
    def __init__(self, Particles, LED, f, dt, FPS):
        self.Particles = Particles
        self.LED = LED
        self.f = f
        self.dt = dt
        self.FPS = FPS
        
    

[0 1 2 3 4 5 6 7 8 0]
